In [ ]:
# IoU 


import numpy as np
def compute_iou(box1, box2, wh=False):
        """
        compute the iou of two boxes.
        Args:
                box1, box2: [xmin, ymin, xmax, ymax] (wh=False) or [xcenter, ycenter, w, h] (wh=True)
                wh: the format of coordinate.
        Return:
                iou: iou of box1 and box2.
        """
        if wh == False:
                xmin1, ymin1, xmax1, ymax1 = box1
                xmin2, ymin2, xmax2, ymax2 = box2
        else:
                xmin1, ymin1 = int(box1[0]-box1[2]/2.0), int(box1[1]-box1[3]/2.0)       # box1 左上角 
                xmax1, ymax1 = int(box1[0]+box1[2]/2.0), int(box1[1]+box1[3]/2.0)       # box1 右下角
                xmin2, ymin2 = int(box2[0]-box2[2]/2.0), int(box2[1]-box2[3]/2.0)       # box2 左上角
                xmax2, ymax2 = int(box2[0]+box2[2]/2.0), int(box2[1]+box2[3]/2.0)       # box2 右下角

        ## 获取矩形框交集对应的左上角和右下角的坐标（intersection）
        xx1 = np.max([xmin1, xmin2])            # 交集框: 左上角取 max
        yy1 = np.max([ymin1, ymin2])
        xx2 = np.min([xmax1, xmax2])            # 交集框: 右下角取 min
        yy2 = np.min([ymax1, ymax2])

        ## 计算两个矩形框面积
        area1 = (xmax1-xmin1) * (ymax1-ymin1) 
        area2 = (xmax2-xmin2) * (ymax2-ymin2)

        inter_area = (np.max([0, xx2-xx1])) * (np.max([0, yy2-yy1]))    #计算交集面积
        iou = inter_area / (area1+area2-inter_area+1e-6)                ＃计算交并比, 注意防止除零
return iou

In [3]:
# 代码联系
import numpy as np

def iou(box1, box2):
    """

    Args:
        box1 (list): [xmin, ymin, xmax, ymax]
        box2 (list): [xmin, ymin, xmax, ymax]
    """
    # 先取两个框的坐标值出来
    xmin1, ymin1, xmax1, ymax1 = box1 
    xmin2, ymin2, xmax2, ymax2 = box2 
    
    # 两个框的面积:
    a1 = (xmax1 - xmin1)*(ymax1 - ymin1)
    a2 = (xmax2 - xmin2)*(ymax2 - ymin2)
    
    # 计算相交框的左上右下坐标
    xx1 = np.max([xmin1, xmin2])   # 左上, 取max, 里面的坐标用min
    yy1 = np.max([ymin1, ymin2])
    xx2 = np.min([xmax1, xmax2])   # 右下, 取min, 里面的坐标用max 
    yy2 = np.min([ymax1, ymax2]) 
    
    # 交集面积
    a_inter = np.max([0, (xx2-xx1)]) * np.max([0, (yy2-yy1)])
    
    # iou
    res = inter_area / (a1+a2 - inter_area + 1e-6)
    
    return res

In [2]:
import numpy as np 

def iou(box1, box2):
    
    # 取坐标值
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2
    
    # 计算两个框的面积
    a1 = (xmax1 - xmin1) * (ymax1 - ymin1)
    a2 = (xmax2 - xmin2) * (ymax2 - ymin2)
    
    # 相交框的左上角和右下角坐标
    xx1 = np.max([xmin1, xmin2])
    yy1 = np.max([ymin1, ymin2])
    xx2 = np.min([xmax1, xmax2])
    yy2 = np.min([ymax1, ymax2])
    
    # 相交框的面积
    inter_area = np.max([0, xx2 - xx1]) * np.max([0, yy2 - yy1])
    
    res = inter_area / (a1 + a2 - inter_area + 1e-6)
    return res

1

# NMS python 版本


In [5]:

## NMS
def py_cpu_nms(dets, thresh): 
    """
    Pure Python NMS baseline.
    本函数只处理单个类别的 nms, 如果dets 里有多个类别, 则直接使用 for 循环去切分处理.
    
    args:
        dets: 某个类的框, x1、y1、x2、y2、以及置信度score
                eg:dets为[[x1,y1,x2,y2,score],[x1,y1,y2,score]……]
        thresh: 是IoU的阈值     
    """ 
    
    x1 = dets[:, 0]         # 注意 dets 是一个矩阵, 所以要取所有行
    y1 = dets[:, 1]
    x2 = dets[:, 2] 
    y2 = dets[:, 3] 
    scores = dets[:, 4] 
    
    #每一个检测框的面积 
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)       # 注意 areas 是一个矩阵
    
    #按照score置信度降序排序 
    order = scores.argsort()[::-1]              # 得到的是降序排列后 score 的下标.   (argsort 默认升序排列)
    
    keep = [] #保留的结果框集合 
    while order.size > 0: 
        i = order[0] 
        keep.append(i)                          # 保留该类剩余box中得分最高的一个 
        
        #得到相交区域,左上及右下  ----  IoU 里常见的操作
        xx1 = np.maximum(x1[i], x1[order[1:]])                  # bboxes[order[1:]] 把第i个之后所有的框都取出来了
        yy1 = np.maximum(y1[i], y1[order[1:]]) 
        xx2 = np.minimum(x2[i], x2[order[1:]]) 
        yy2 = np.minimum(y2[i], y2[order[1:]]) 
        
        #计算相交的面积,不重叠时面积为0 
        w = np.maximum(0.0, xx2 - xx1 + 1) 
        h = np.maximum(0.0, yy2 - yy1 + 1) 
        inter = w * h 
        
        #计算IoU：重叠面积 /（面积1+面积2-重叠面积） 
        ovr = inter / (areas[i] + areas[order[1:]] - inter)     # areas[order[1:]] 会得到一个矩阵, 它与areas[i]相加时会自动广播, 然后减去inter (一个数) 就是普通的矩阵减法
        
        #保留IoU小于阈值的box 
        inds = np.where(ovr <= thresh)[0]                       # inds 是被保留下来的框对应在 ovr 里的 index
        order = order[inds + 1]                                 # 因为ovr数组的长度比order数组少一个, 所以这里要将所有下标后移一位 
        
    return keep

In [6]:
import numpy as np

def NMS(bboxes, scores, threshold):
    """
    :param bboxes: numpy数组，形状为(N, 4)，表示检测框的坐标。
    :param scores: numpy数组，形状为(N,)，表示每个检测框的得分。
    :param threshold: 阈值，用来控制NMS的筛选效果。
    :return: 选出的检测框的坐标和得分。
    """
    # 获取所有检测框的面积
    areas = (bboxes[:, 2] - bboxes[:, 0] + 1) * (bboxes[:, 3] - bboxes[:, 1] + 1)

    # 根据检测框得分降序排列
    order = scores.argsort()[::-1]

    # 选出所有得分大于等于阈值的检测框
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        # 计算当前检测框与其余检测框的交并比
        x1 = np.maximum(bboxes[i, 0], bboxes[order[1:], 0])         # bboxes[order[1:]] 把第i个之后所有的框都取出来了
        y1 = np.maximum(bboxes[i, 1], bboxes[order[1:], 1])
        x2 = np.minimum(bboxes[i, 2], bboxes[order[1:], 2])
        y2 = np.minimum(bboxes[i, 3], bboxes[order[1:], 3])
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter)         # areas[order[1:]] 会得到一个矩阵, 它与areas[i]相加时会自动广播, 然后减去inter (一个数) 就是普通的矩阵减法
        
        # 将所有与当前检测框的交并比大于阈值的检测框从序列中删除
        inds = np.where(iou <= threshold)[0]
        order = order[inds + 1]
    
    return keep

# 使用方法：


# python
# 假设已经有了检测框坐标和得分
bboxes = np.array([[10, 20, 50, 100], [20, 30, 100, 200], [50, 70, 200, 300]])
scores = np.array([0.8, 0.9, 0.7])

# 调用NMS函数
keep = NMS(bboxes, scores, threshold=0.1)

# 根据选中的检测框坐标和得分继续后续处理
selected_bboxes = bboxes[keep]
selected_scores = scores[keep]

print(selected_bboxes, "\n", selected_scores)

[[ 20  30 100 200]] 
 [0.9]
